## Computer Vision Algorithm for Tracking Planarian Motion
developed by Hokin Deng xueqiandeng@yahoo.com

# Import Dependencies and Data

Import packages and make sure of the python technicality

In [1]:
import sys
import os
print("Python Version:", sys.version)
print("Python Executable:", sys.executable)
print("Python Path:", sys.path)

Python Version: 3.11.4 (main, Jul  5 2023, 08:41:25) [Clang 14.0.6 ]
Python Executable: /Users/billdeng/anaconda3/envs/unlearning_Version1/bin/python
Python Path: ['/Users/billdeng/PycharmProjects/unicellular', '/Users/billdeng/PycharmProjects/unicellular', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python311.zip', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python3.11', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python3.11/lib-dynload', '', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python3.11/site-packages']


In [2]:
import cv2
import math
import copy
import numpy as np
from collections import deque
import numpy as np
import argparse
import imutils
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(threshold=50)
import queue
import math

The video data should be named as "sample.avi" and put in the folder as the notebook.

In [3]:
# Open a video file (replace 'sample.avi' with the path)
cap = cv2.VideoCapture('sample.avi')

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

Display the video if want to have a look at it

In [4]:
#cv2.startWindowThread()
# Loop to read and display frames
#while True:
    # Read a frame from the video
#    ret, frame = cap.read()
    # If the video has ended, break out of the loop
#    if not ret:
#        break
    # Display the frame in a window
#    cv2.imshow('Video', frame)
    # Exit the loop if the 'q' key is pressed
#    if cv2.waitKey(25) & 0xFF == ord('q'):
#        break

# Release the video capture object and close the window

First, get basic properties about our video

In [5]:
# Get basic video properties
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
video_duration_sec = frame_count / frame_rate if frame_rate > 0 else 0

# Print the video properties
print(f"Frame Count: {frame_count}")
print(f"Frame Width: {frame_width}")
print(f"Frame Height: {frame_height}")
print(f"Frame Rate: {frame_rate} frames per second")
print(f"Video Duration: {video_duration_sec:.2f} seconds")

Frame Count: 1202
Frame Width: 2160
Frame Height: 2160
Frame Rate: 10.0 frames per second
Video Duration: 120.20 seconds


In [6]:
example_ret, example_frame = cap.read()
print( 'ret', type(example_ret))
print( 'frame', type(example_frame))
print('ret', example_ret)
print('frame', example_frame)

ret <class 'bool'>
frame <class 'numpy.ndarray'>
ret True
frame [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


In [7]:
# frame_number = 100  # The frame you want to access
# Set the video position to the desired frame
# cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number - 1)
# Read the frame
# ret, frame = cap.read()
# if ret:
    # Process the frame
#     cv2.imshow('Frame 100', frame)
#    cv2.waitKey(0)  # Wait for a key press to close the image window
# else:
#    print("Error: Unable to read the frame")

Inspect function for a frame

In [8]:
def inspect_frame_from_video(cap_for_here, frame_number):
    """
    Inspects a specific frame in a video.

    :param cap_for_here: video reference
    :param frame_number: The frame number to inspect (1-based index).
    :return: None
    """
    # Open the video file
    if not cap_for_here.isOpened():
        print("Error: Unable to open video file")
        return
    # Check if the frame number is valid
    total_frames = cap_for_here.get(cv2.CAP_PROP_FRAME_COUNT)
    if frame_number < 1 or frame_number > total_frames:
        print(f"Frame number should be between 1 and {int(total_frames)}")
        return
    # Set the video position to the desired frame (0-based index for frame_number)
    cap_for_here.set(cv2.CAP_PROP_POS_FRAMES, frame_number - 1)
    # Read the frame
    ret_temp, frame_temp = cap_for_here.read()
    if ret_temp:
        # Display the frame
        # cv2.imshow(f'Frame {frame_number}', frame)
        # cv2.waitKey(0)  # Wait for a key press to close the image window
        # cv2.destroyAllWindows()
        frame_h, frame_w = frame_temp.shape[:2]
        print(f"Frame Width: {frame_h}")
        print(f"Frame Height: {frame_w}")
        # Color Channels
        channels = frame_temp.shape[2] if len(frame_temp.shape) == 3 else 1
        print(f"Color Channels: {channels}")
        # Data Type
        data_type = frame_temp.dtype
        print(f"Data Type: {data_type}")
        # Aspect Ratio
        aspect_ratio = frame_w / frame_h
        print(f"Aspect Ratio: {aspect_ratio}")
        # Resolution (assuming a standard display resolution of 96 PPI)
        # Color Space (assuming default BGR)
        color_space = "BGR" if channels == 3 else "Grayscale"
        print(f"Color Space: {color_space}")
        # Histogram for each channel
        if channels > 1:
            for i, col in enumerate(['Blue', 'Green', 'Red']):
                hist = cv2.calcHist([frame_temp], [i], None, [256], [0, 256])
                print(f"Histogram for {col} channel: {np.array(hist).flatten()}")
        else:
            hist = cv2.calcHist([frame_temp], [0], None, [256], [0, 256])
            print(f"Histogram for Grayscale: {np.array(hist).flatten()}")
    else:
        print("Error: Unable to read the frame")

In [9]:
def inspect_given_a_frame(this_frame):
    frame_h, frame_w = this_frame.shape[:2]
    print(f"Frame Width: {frame_h}")
    print(f"Frame Height: {frame_w}")
    channels = this_frame.shape[2] if len(this_frame.shape) == 3 else 1
    print(f"Color Channels: {channels}")
    data_type = this_frame.dtype
    print(f"Data Type: {data_type}")
    aspect_ratio = frame_w / frame_h
    print(f"Aspect Ratio: {aspect_ratio}")
    color_space = "BGR" if channels == 3 else "Grayscale"
    print(f"Color Space: {color_space}")

In [10]:
# Usage example
video_file = 'sample.avi'
frame_to_inspect = 100  # Adjust the frame number as needed
inspect_frame_from_video(cap, frame_to_inspect)

Frame Width: 2160
Frame Height: 2160
Color Channels: 3
Data Type: uint8
Aspect Ratio: 1.0
Color Space: BGR
Histogram for Blue channel: [      0.       0.       0. ...   31073.   57326. 3775011.]
Histogram for Green channel: [      0.       0.       0. ...   31073.   57326. 3775011.]
Histogram for Red channel: [      0.       0.       0. ...   31073.   57326. 3775011.]


In [11]:
ret, frame_for_use = cap.read()
inspect_given_a_frame(frame_for_use)
cap.release()
cv2.destroyAllWindows()

Frame Width: 2160
Frame Height: 2160
Color Channels: 3
Data Type: uint8
Aspect Ratio: 1.0
Color Space: BGR


## Pre-Processing


Background subtraction

In [ ]:
i = 0
# Initialize video capture
cap = cv2.VideoCapture('sample.avi')
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('background_remove.avi', fourcc, frame_rate, (frame_width, frame_height), False)
backSub = cv2.createBackgroundSubtractorMOG2()
while True:
    i += 1
    ret, frame = cap.read()
    if not ret:
        break
    fgMask = backSub.apply(frame)
    out.write(fgMask)
    print(f'\rProgress: {i}', end='')
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

Spatial Smoothing

In [ ]:
i = 0
cap = cv2.VideoCapture('background_remove.avi')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('background_remove_spatial_smoothed.avi', fourcc, frame_rate, (frame_width, frame_height), False)
while True:
    i += 1
    ret, frame = cap.read()
    if not ret:
        break
    blurred_frame = cv2.GaussianBlur(frame, (5, 5), 0)
    out.write(blurred_frame)
    print(f'\rProgress: {i}', end='')
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

Temporal Smoothing

In [ ]:
cap = cv2.VideoCapture('background_remove_spatial_smooth.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID') # You can also use 'XVID'
out = cv2.VideoWriter('b_r_s_s_t_s.avi', fourcc, frame_rate, (frame_width, frame_height), False)
buffer_size = 5
frame_buffer = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_buffer.append(frame)
    if len(frame_buffer) > buffer_size:
        frame_buffer.pop(0)
    # Temporal smoothing (average of frames in buffer)
    temp_smoothed = np.mean(frame_buffer, axis=0).astype(np.uint8)
    # Write frame to video
    out.write(temp_smoothed)
    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

do everything all at once

In [ ]:
# Initialize video capture
cap = cv2.VideoCapture('sample.avi')
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can also use 'XVID'
out = cv2.VideoWriter('pre_dense.mp4', fourcc, frame_rate, (frame_width, frame_height), False)
# Background subtractor
backSub = cv2.createBackgroundSubtractorMOG2()
# Buffer for temporal smoothing
buffer_size = 5
frame_buffer = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Background subtraction
    fgMask = backSub.apply(frame)
    # Spatial smoothing (Gaussian blur)
    blurred = cv2.GaussianBlur(fgMask, (5, 5), 0)
    # Add frame to buffer for temporal smoothing
    frame_buffer.append(blurred)
    if len(frame_buffer) > buffer_size:
        frame_buffer.pop(0)
    # Temporal smoothing (average of frames in buffer)
    temp_smoothed = np.mean(frame_buffer, axis=0).astype(np.uint8)
    # Write frame to video
    out.write(temp_smoothed)
    # Display result
    # cv2.imshow('Frame', frame)
    # cv2.imshow('FG Mask', fgMask)
    # cv2.imshow('Blurred', blurred)
    # cv2.imshow('Temporally Smoothed', temp_smoothed)
    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

dense opti flow
this takes a very very long time

In [ ]:
cap = cv2.VideoCapture("pre_dense.mp4")
if not cap.isOpened():
    print("Error opening video file")
    exit()
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Try 'XVID' if 'mp4v' does not work
out = cv2.VideoWriter('dense_opti_flow_v2.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
ret, first_frame = cap.read()
if not ret:
    print("Error reading first frame")
    cap.release()
    exit()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(first_frame)
mask[..., 1] = 255
i = 0
while True:
    i += 1
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    mask[..., 0] = angle * 180 / np.pi / 2
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    out.write(rgb)
    print(f'\rProgress: {i}', end='')
    prev_gray = gray
cap.release()
out.release()
cv2.destroyAllWindows()

segment each worm in dense optical flow video

In [ ]:
cap = cv2.VideoCapture('dense_opti_flow_v2.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Try 'XVID' if 'mp4v' does not work
out = cv2.VideoWriter('dense_opt_segmented.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
# Read the first frame
ret, frame1 = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
j = 0
while True:
    j += 1
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 3  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    # Segment mask based on connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(motion_mask), connectivity=8)
    min_area = 50  # Minimum area for connected components
    # Draw bounding boxes around components
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Display the result
    out.write(frame2)
    # cv2.imshow('Segmented Frame', frame2)
    # Update previous frame
    print(f'\rProgress: {j}', end='')
    prvs = next
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

segment each worm without optical flow processing

In [ ]:
cap = cv2.VideoCapture('pre_dense.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Try 'XVID' if 'mp4v' does not work
out = cv2.VideoWriter('predense_degmented.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
# Read the first frame
ret, frame1 = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
j = 0
while True:
    j += 1
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 3  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    # Segment mask based on connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(motion_mask), connectivity=8)
    min_area = 50  # Minimum area for connected components
    # Draw bounding boxes around components
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Display the result
    out.write(frame2)
    # cv2.imshow('Segmented Frame', frame2)
    # Update previous frame
    print(f'\rProgress: {j}', end='')
    prvs = next
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

Define a function for video to image and use it

In [ ]:
# Define a function to extract and save frames from a video file
current_location = os.getcwd();
output_folder = current_location + '/raw_images'
video_path = current_location + '/sample.avi'

def video2image(video_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the video
    video = cv2.VideoCapture(video_path)

    # Initialize frame count
    count = 0

    # Iterate through video frames
    while True:
        # Read a frame
        success, frame = video.read()
        # Break if no frame is read (end of video)
        if not success:
            break
        # Save the frame as an image
        cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), frame)
        # Increment frame count
        count += 1

    # Release the video object
    video.release()

    return count

Save the video into images

In [ ]:
frame_count = video2image(video_path, output_folder)

Design a crop function that remove the dish

In [ ]:
def crop_circle(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # Create a mask with the same dimensions as the image
    mask = np.zeros_like(image)
    rows, cols, _ = mask.shape
    # Compute the center and radius of the circle
    center = (cols // 2, rows // 2)
    radius = min(center[0], center[1], rows - center[1], cols - center[0])
    # Draw the circular mask
    cv2.circle(mask, center, radius, (255, 255, 255), -1)
    # Apply the mask
    circular_image = cv2.bitwise_and(image, mask)
    # Optionally, you can remove the black background
    masked_data = cv2.cvtColor(circular_image, cv2.COLOR_BGR2BGRA)
    masked_data[mask == 0] = [0, 0, 0, 0]
    # Save or display the result
    cv2.imwrite('circular_image.png', masked_data)
    # cv2.imshow('Circular Image', masked_data)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

threshold the original video and set the threshold

In [ ]:
thresh = 127

apply to original video

In [ ]:
# Load the video
cap = cv2.VideoCapture('sample.avi')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Try 'XVID' if 'mp4v' does not work
out = cv2.VideoWriter('threshold_original.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
# Check if video opened successfully
if not cap.isOpened():
    print("Error opening video file")
# Read until video is completed
j = 0
while cap.isOpened():
    j += 1
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        # Convert to grayscale
        # Apply threshold for each channel
        _, red_channel = cv2.threshold(frame[:,:,0], thresh, 255, cv2.THRESH_BINARY)
        _, green_channel = cv2.threshold(frame[:,:,1], thresh, 255, cv2.THRESH_BINARY)
        _, blue_channel = cv2.threshold(frame[:,:,2], thresh, 255, cv2.THRESH_BINARY)
    # Combine the channels back
        thresh_frame = cv2.merge([red_channel, green_channel, blue_channel])
        # Display the resulting frame
        out.write(thresh_frame)
        # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        print(f'\rProgress: {j}', end='')
    else:
        break
# When everything done, release the video capture object
cap.release()
out.release()
cv2.destroyAllWindows()

apply erode and dilate

In [ ]:
# Load the video
cap = cv2.VideoCapture('pre_dense.mp4')
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Try 'XVID' if 'mp4v' does not work
out = cv2.VideoWriter('pre_dense_erode_dilate.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
j = 0
while True:
    j += 1
    # Read each frame
    ret, frame = cap.read()
    if not ret:
        break
    # Apply erosion and then dilation
    eroded_frame = cv2.erode(frame, kernel, iterations=1)
    dilated_frame = cv2.dilate(eroded_frame, kernel, iterations=1)
    # Display the processed frame
    out.write(dilated_frame)
    print(f'\rProgress: {j}', end='')
    # Break the loop with a key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
out.release()

motion segmentation using connectivity after erode and dilate

In [ ]:
cap = cv2.VideoCapture('pre_dense_erode_dilate.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('pre_dense_erode_dilate_segmented.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
# Read the first frame
ret, frame1 = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
j = 0
while True:
    j += 1
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 1  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    cv2.imshow('Segmented Frame', motion_mask)
    # Segment mask based on connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(motion_mask), connectivity=8)
    min_area = 300  # Minimum area for connected components
    # Draw bounding boxes around components
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # cv2.imshow('Segmented Frame', frame2)
    # Display the result
    out.write(frame2)
    # Update previous frame
    print(f'\rProgress: {j}', end='')
    prvs = next
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

erode and dilate also motion mask, save motion mask actually

In [ ]:
cap = cv2.VideoCapture('pre_dense_erode_dilate.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('motion_mask_segmented.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
# Read the first frame
ret, frame1 = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
j = 0
while True:
    j += 1
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 1  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
    dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
    # cv2.imshow('Segmented Frame', dilated_frame)
    # Segment mask based on connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
    min_area = 300  # Minimum area for connected components
    # Draw bounding boxes around components
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
    save_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    # cv2.imshow('Segmented Frame', dilated_frame)
    # cv2.imshow('Segmented Frame', frame2)
    # Display the result
    rgb_frame = cv2.cvtColor(save_frame, cv2.COLOR_GRAY2RGB)
    # inspect_given_a_frame(frame2)
    # inspect_given_a_frame(rgb_frame)
    out.write(save_frame)
    # Update previous frame
    print(f'\rProgress: {j}', end='')
    prvs = next
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('pre_dense_erode_dilate.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('motion_mask_segmented_v2.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
# Read the first frame
ret, frame1 = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
j = 0
while True:
    j += 1
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 1  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
    dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
    # cv2.imshow('Segmented Frame', dilated_frame)
    # Segment mask based on connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
    min_area = 300  # Minimum area for connected components
    # Draw bounding boxes around components
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
    t_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    # cv2.imshow('Segmented Frame', dilated_frame)
    # cv2.imshow('Segmented Frame', frame2)
    # Display the result
    rgb_frame = cv2.cvtColor(t_frame, cv2.COLOR_GRAY2RGB)
    # inspect_given_a_frame(frame2)
    # inspect_given_a_frame(rgb_frame)
    # inspect_given_a_frame(rgb_frame)
    out.write(rgb_frame)
    # Update previous frame
    print(f'\rProgress: {j}', end='')
    prvs = next
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

dilate even more for segmentation

## Overlay

In [ ]:
original_cap = cv2.VideoCapture('sample.avi')
cap = cv2.VideoCapture('pre_dense_erode_dilate.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (45, 45))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('overlay.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
# Read the first frame
ret, frame1 = cap.read()
not_useful, ori_frame = original_cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
j = 0
while True:
    j += 1
    ret, frame2 = cap.read()
    not_useful, ori_frame = original_cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 1  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
    dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
    min_area = 300
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
    temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
    out.write(ori_frame)
    print(f'\rProgress: {j}', end='')
    prvs = next
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

## Segmentation

Initialize Paths

In [ ]:
current_location = os.getcwd();
output_folder = current_location + '/worm_segmentation'
video_path = current_location + '/sample.avi'

Initialize input videos and output videos

In [ ]:
motion_mask_cap = cv2.VideoCapture('pre_dense_erode_dilate.mp4')
original_video_cap = cv2.VideoCapture('sample.avi')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (45, 45))
f_worm_1 = cv2.VideoWriter_fourcc(*'mp4v')
f_worm_2 = cv2.VideoWriter_fourcc(*'mp4v')
f_worm_3 = cv2.VideoWriter_fourcc(*'mp4v')
f_worm_4 = cv2.VideoWriter_fourcc(*'mp4v')
f_worm_5 = cv2.VideoWriter_fourcc(*'mp4v')
f_worm_6 = cv2.VideoWriter_fourcc(*'mp4v')
f_worm_7 = cv2.VideoWriter_fourcc(*'mp4v')
f_worm_8 = cv2.VideoWriter_fourcc(*'mp4v')
f_worm_9 = cv2.VideoWriter_fourcc(*'mp4v')
f_worm_10 = cv2.VideoWriter_fourcc(*'mp4v')
worm_1_out = cv2.VideoWriter(output_folder + '/worm_1_out.mp4', f_worm_1, frame_rate, (frame_width, frame_height), True)
worm_2_out = cv2.VideoWriter(output_folder + '/worm_2_out.mp4', f_worm_2, frame_rate, (frame_width, frame_height), True)
worm_3_out = cv2.VideoWriter(output_folder + '/worm_3_out.mp4', f_worm_3, frame_rate, (frame_width, frame_height), True)
worm_4_out = cv2.VideoWriter(output_folder + '/worm_4_out.mp4', f_worm_4, frame_rate, (frame_width, frame_height), True)
worm_5_out = cv2.VideoWriter(output_folder + '/worm_5_out.mp4', f_worm_5, frame_rate, (frame_width, frame_height), True)
worm_6_out = cv2.VideoWriter(output_folder + '/worm_6_out.mp4', f_worm_6, frame_rate, (frame_width, frame_height), True)
worm_7_out = cv2.VideoWriter(output_folder + '/worm_7_out.mp4', f_worm_7, frame_rate, (frame_width, frame_height), True)
worm_8_out = cv2.VideoWriter(output_folder + '/worm_8_out.mp4', f_worm_8, frame_rate, (frame_width, frame_height), True)
worm_9_out = cv2.VideoWriter(output_folder + '/worm_9_out.mp4', f_worm_9, frame_rate, (frame_width, frame_height), True)
worm_10_out = cv2.VideoWriter(output_folder + '/worm_10_out.mp4', f_worm_10, frame_rate, (frame_width, frame_height), True)

clean up make sure all number is good

In [ ]:
total_it_number = 0

if not motion_mask_cap.isOpened():
    print("Error: Could not open motion mask.")
else:
    # Get the total number of frames in the video
    total_it_number = int(motion_mask_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total number of frames in the motion mask: {total_it_number}")

if not original_video_cap.isOpened():
    print("Error: Could not open motion mask.")
else:
    # Get the total number of frames in the video
    total_it_number = int(original_video_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total number of frames in the original video: {total_it_number}")

print(f"to process the video need to iterate over: {total_it_number}")

In [ ]:
# Read the first motion
ret, motion_frame_init = motion_mask_cap.read()
if not ret:
    print("Failed to read the motion ask")
    cap.release()
else:
    print("motion frame read")
print("Shape of Frame", motion_frame_init.shape)
print("First Row", motion_frame_init[:,1])
previous_motion_frame = cv2.cvtColor(motion_frame_init, cv2.COLOR_BGR2GRAY)
inspect_given_a_frame(previous_motion_frame)
print("Shape of Frame", previous_motion_frame.shape)
print("First Row", previous_motion_frame[:,1].shape)

pilot the code a bit

In [ ]:
for j in range(total_it_number):
    ret, temporal_motion_frame = motion_mask_cap.read()
    if not ret:
        break
    current_motion_frame = cv2.cvtColor(temporal_motion_frame, cv2.COLOR_BGR2GRAY)
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(previous_motion_frame, current_motion_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 1  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
    dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
    print('Number of labels', num_labels, '\n')
    print('labels', labels, '\n')
    print('stats', stats, '\n')
    print('centroids', centroids, '\n')
    min_area = 5000
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
    temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
    cv2.imshow('Segmented Frame', rgb_frame)
    out.write(rgb_frame)
    print(f'\rProgress: {j}', end='')
    prvs = next
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
tye = num_labels, labels, stats, centroids

In [ ]:
num_la, la, st, cent = tye

In [ ]:
print(type(num_la))

In [ ]:
print(centroids[2][0])


In [ ]:
print(st)

In [ ]:
print(st[1][0])

In [ ]:
print(centroids.shape)

In [ ]:
stats_2 = stats
print(stats_2)

In [ ]:
hum = stats

In [ ]:
hum = np.append(hum, stats_2)
print(hum)

make an object array and work with that

In [ ]:
hum = np.array(None,dtype=object)
print(hum)
hum = np.append(hum, stats_2)
print(hum)
hum = np.append(hum, stats_2)
print(hum)

In [ ]:
print(tye[0])

In [ ]:
print(tye[1])

In [ ]:
print(tye[2])

In [ ]:
print(tye[3])

# Construction of a Worm Object
## the idea is that, to make a worm object, this object should be able to take each frame as input,
## find the motion mask to correspond to the worm, and append to the worm object

In [ ]:
class Worm:
    def __init__(self, worm_index, original_cap, label_history, stats_history, centroid_history, another_variable_in_case=None):
        self.worm_index = worm_index
        self.original_cap = original_cap
        self.label_history = label_history
        self.stats_history = stats_history
        self.centroid_history = centroid_history
        self.another_variable_in_case = another_variable_in_case

    def display_info(self):
        info = f"worn index: {self.worm_index}, video_cap: {self.original_cap}, label_history: {self.label_history}, stats_history: {self.stats_history}, centroid_history: {self.centroid_history}"
        print(info)

    def update_another_variable(self, another_variable_in_case):
        """Update the car's mileage."""
        if another_variable_in_case >= self.another_variable_in_case:
            self.another_variable_in_case = another_variable_in_case
        else:
            print("Error: another_variable_in_case cannot be reduced.")

# Pilot with Single Worm first before doing 10

Basic set up

In [ ]:
current_location = os.getcwd();
output_folder = current_location + '/worm_segmentation'
video_path = current_location + '/sample.avi'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
motion_mask_cap = cv2.VideoCapture('pre_dense_erode_dilate.mp4')
original_video_cap = cv2.VideoCapture('sample.avi')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (45, 45))
f_worm_1 = cv2.VideoWriter_fourcc(*'mp4v')
worm_1_out = cv2.VideoWriter(output_folder + '/worm_1_out.mp4', f_worm_1, frame_rate, (frame_width, frame_height), True)

loop through the first 1/10 of the video

In [ ]:
total_it_number = 0

if not motion_mask_cap.isOpened():
    print("Error: Could not open motion mask.")
else:
    # Get the total number of frames in the video
    total_it_number = int(motion_mask_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total number of frames in the motion mask: {total_it_number}")

if not original_video_cap.isOpened():
    print("Error: Could not open motion mask.")
else:
    # Get the total number of frames in the video
    total_it_number = int(original_video_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total number of frames in the original video: {total_it_number}")

print(f"to process the video need to iterate over: {total_it_number}")

total_it_number = int(total_it_number / 10)

print(f"to process the video need to iterate over: {total_it_number}")

actually, construct the worm object before looping, that means, I need to make the first frame

In [ ]:
# Read the first motion
ret, motion_frame_init = motion_mask_cap.read()
if not ret:
    print("Failed to read the motion ask")
    cap.release()
else:
    print("motion frame read")
print("Shape of Frame", motion_frame_init.shape)
print("First Row", motion_frame_init[:,1])
previous_motion_frame = cv2.cvtColor(motion_frame_init, cv2.COLOR_BGR2GRAY)
inspect_given_a_frame(previous_motion_frame)
print("Shape of Frame", previous_motion_frame.shape)
print("First Row", previous_motion_frame[:,1].shape)

for worm initialization, let us analyze frame by frame

In [ ]:
ret, current_motion_frame = motion_mask_cap.read()
current_motion_frame = cv2.cvtColor(current_motion_frame, cv2.COLOR_BGR2GRAY)
# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(previous_motion_frame, current_motion_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
print('Number of labels', num_labels, '\n')
print('labels', labels, '\n')
print('stats', stats, '\n')
print('centroids', centroids, '\n')
min_area = 5000
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
previous_motion_frame = current_motion_frame

In [ ]:
print(sum(sum(labels)))
print('Number of labels', num_labels)

# Create the heatmap
plt.imshow(labels, cmap='hot', interpolation='nearest')

# Adding a colorbar
plt.colorbar()

# Adding titles and labels (optional)
plt.title('Heatmap of a 2D NumPy Array')
plt.xlabel('X-axis Label')
plt.ylabel('Y-axis Label')

# Show the plot
plt.show()

let us look at next frame

In [ ]:
ret, current_motion_frame = motion_mask_cap.read()
current_motion_frame = cv2.cvtColor(current_motion_frame, cv2.COLOR_BGR2GRAY)
# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(previous_motion_frame, current_motion_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
print('Number of labels', num_labels, '\n')
print('labels', labels, '\n')
print('stats', stats, '\n')
print('centroids', centroids, '\n')
min_area = 5000
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
previous_motion_frame = current_motion_frame

In [ ]:
print(sum(sum(labels)))
print('Number of labels', num_labels)

# Create the heatmap
plt.imshow(labels, cmap='hot', interpolation='nearest')

# Adding a colorbar
plt.colorbar()

# Adding titles and labels (optional)
plt.title('Heatmap of a 2D NumPy Array')
plt.xlabel('X-axis Label')
plt.ylabel('Y-axis Label')

# Show
plt.show()

standard initialization

In [ ]:
# reset cap
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
# Read the first motion
ret, motion_frame_init = motion_mask_cap.read()
if not ret:
    print("Failed to read the motion ask")
    cap.release()
else:
    print("motion frame read")
print("Shape of Frame", motion_frame_init.shape)
print("First Row", motion_frame_init[:,1])
previous_motion_frame = cv2.cvtColor(motion_frame_init, cv2.COLOR_BGR2GRAY)
inspect_given_a_frame(previous_motion_frame)
print("Shape of Frame", previous_motion_frame.shape)
print("First Row", previous_motion_frame[:,1].shape)

loop through the video

In [ ]:
for j in range(total_it_number):
    ret, temporal_motion_frame = motion_mask_cap.read()
    if not ret:
        break
    current_motion_frame = cv2.cvtColor(temporal_motion_frame, cv2.COLOR_BGR2GRAY)
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(previous_motion_frame, current_motion_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 1  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
    dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
    print('Number of labels', num_labels, '\n')
    print('labels', labels, '\n')
    print('stats', stats, '\n')
    print('centroids', centroids, '\n')
    min_area = 5000










    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
    temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
    cv2.imshow('Segmented Frame', rgb_frame)
    out.write(rgb_frame)
    print(f'\rProgress: {j}', end='')
    prvs = next
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

## Cut to 1/4 of the original for easy processing

In [ ]:
# Initialize video capture
cap = cv2.VideoCapture('sample.avi')
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('shorter.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
for i in range(int(frame_count/4)):
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)
    print(f'\rProgress: {i}', end='')
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

## Background Subtraction, Spatial Smoothing, Temporal Smoothing

In [ ]:
# Initialize video capture
cap = cv2.VideoCapture('shorter.mp4')
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can also use 'XVID'
out = cv2.VideoWriter('processed_v1.mp4', fourcc, frame_rate, (frame_width, frame_height), False)
# Background subtractor
backSub = cv2.createBackgroundSubtractorMOG2()
# Buffer for temporal smoothing
buffer_size = 5
frame_buffer = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Background subtraction
    fgMask = backSub.apply(frame)
    # Spatial smoothing (Gaussian blur)
    blurred = cv2.GaussianBlur(fgMask, (5, 5), 0)
    # Add frame to buffer for temporal smoothing
    frame_buffer.append(blurred)
    if len(frame_buffer) > buffer_size:
        frame_buffer.pop(0)
    # Temporal smoothing (average of frames in buffer)
    temp_smoothed = np.mean(frame_buffer, axis=0).astype(np.uint8)
    # Write frame to video
    out.write(temp_smoothed)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

## Erode and Dilate

In [ ]:
# Load the video
cap = cv2.VideoCapture('processed_v1.mp4')
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('processed_v2.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
j = 0
while True:
    j += 1
    # Read each frame
    ret, frame = cap.read()
    if not ret:
        break
    # Apply erosion and then dilation
    eroded_frame = cv2.erode(frame, kernel, iterations=1)
    dilated_frame = cv2.dilate(eroded_frame, kernel, iterations=1)
    # Display the processed frame
    out.write(dilated_frame)
    print(f'\rProgress: {j}', end='')
    # Break the loop with a key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
out.release()

## Overlay

In [ ]:
original_cap = cv2.VideoCapture('shorter.mp4')
cap = cv2.VideoCapture('processed_v2.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (75, 75))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('processed_v3.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
# Read the first frame
ret, frame1 = cap.read()
not_useful, ori_frame = original_cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
j = 0
while True:
    j += 1
    ret, frame2 = cap.read()
    not_useful, ori_frame = original_cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 1  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
    dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
    min_area = 300
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
    temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
    out.write(ori_frame)
    print(f'\rProgress: {j}', end='')
    prvs = next
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

## Worm Object

## Make Minimum Area a Global Parameter

In [12]:
min_area = 2000

# Helper Function

In [13]:
def create_list_of_n_objects(n):
    """
    Create a list of n objects, each object is its index in the list.
    """
    return [i for i in range(n)]

# Example usage
n = 10
list_of_n_objects = create_list_of_n_objects(n)

def iterate_and_print_list(input_list):
    """
    Iteratively list all objects in the provided list.
    """
    for item in input_list:
        print(item)

# Example usage with a list of 10 items
iterate_and_print_list(list_of_n_objects)

0
1
2
3
4
5
6
7
8
9


In [14]:
def check_overlap(x1, y1, width1, height1, x2, y2, width2, height2):
    # Check if one square is to the left of the other
    if x1 + width1 <= x2 or x2 + width2 <= x1:
        return False
    # Check if one square is above the other
    if y1 + height1 <= y2 or y2 + height2 <= y1:
        return False
    # If neither condition is true, squares must be overlapping
    return True

# Example usage
x1, y1, width1, height1 = 10, 10, 30, 30  # Square 1
x2, y2, width2, height2 = 20, 20, 30, 30  # Square 2

overlap = check_overlap(x1, y1, width1, height1, x2, y2, width2, height2)
print("Do the squares overlap?", overlap)

Do the squares overlap? True


In [15]:
def get_nth_item(q, n):
    if n >= q.qsize():
        return None  # or raise an exception if you prefer

    for _ in range(n - 1):
        q.get()

    nth_item = q.get()
    return nth_item

# Example
q = queue.Queue()
q.put('item1')
q.put('item2')
q.put('item3')

nth_item = get_nth_item(q, 2)  # Get the 2nd item
print("The 2nd item in the queue is:", nth_item)

The 2nd item in the queue is: item2


In [16]:
def distance_between_points(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Example usage
x1, y1 = 1, 2
x2, y2 = 4, 6
distance = distance_between_points(x1, y1, x2, y2)
print("The distance between the points is:", distance)

The distance between the points is: 5.0


In [17]:
def white_outside_rectangle(fra, x, y, width, height):
    """
    Makes everything outside a specified rectangle in the frame purely white.
    :param fra: The original image/frame.
    :param x: The x-coordinate of the top-left corner of the rectangle.
    :param y: The y-coordinate of the top-left corner of the rectangle.
    :param width: The width of the rectangle.
    :param height: The height of the rectangle.
    :return: Modified frame with white outside the specified rectangle.
    """
    # Make a copy of the frame
    modified_frame = np.copy(fra)
    # Draw white rectangles over the areas outside the specified rectangle
    # Top
    modified_frame[0:y, :] = 255
    # Bottom
    modified_frame[y+height:fra.shape[0], :] = 255
    # Left
    modified_frame[:, 0:x] = 255
    # Right
    modified_frame[:, x+width:fra.shape[1]] = 255
    return modified_frame
# Example usage
# frame = cv2.imread('path_to_your_image.jpg')  # Load your frame or image
# modified_frame = white_outside_rectangle(frame, 50, 50, 100, 100)
# cv2.imshow('Modified Frame', modified_frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Main Code

In [18]:
current_location = os.getcwd();
output_folder = current_location + '/pilot_images'
monitor_folder = current_location + '/monitor_images'
video_path = current_location + '/sample.avi'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
if not os.path.exists(monitor_folder):
    os.makedirs(monitor_folder)

class Worm:
    def __init__(self, worm_index, original_Cap, init_label, init_stats,init_centroid,another_variable_in_case=None):
        self.worm_index = worm_index
        self.original_cap = original_Cap
        self.current_label = init_label
        self.current_stats = init_stats
        self.current_centroid = init_centroid
        self.label_history = queue.Queue()
        self.stats_history = queue.Queue()
        self.centroid_history = queue.Queue()
        self.another_variable_in_case = another_variable_in_case
        # also establish the output flow
        self.frame_width = int(original_Cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(original_Cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.frame_rate = original_Cap.get(cv2.CAP_PROP_FPS)
        cur_location = os.getcwd()
        text = '/worm_' + str(worm_index)
        self.output_folder = cur_location + text
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

    def display_info(self):
        info = f"worn index: {self.worm_index}, video_cap: {self.original_cap}, label_history: {self.label_history}, stats_history: {self.stats_history}, centroid_history: {self.centroid_history}, \n"
        print(info)
        info = f"current label: {self.current_label}, current_stats: {self.current_stats}, current_centroid: {self.current_centroid}, stats_history: {self.stats_history}, centroid_history: {self.centroid_history}, \n"
        print(info)
        print('Current Stats \n')
        print(self.current_stats)
        print('\n')
        print('\n')
        print('Current Label \n')
        print(self.current_label)
        print('\n')
        print('\n')
        print('Current Centroid\n')
        print(self.current_centroid)
        print('\n')
        print('\n')

    def update_another_variable(self, another_variable_in_case):
        """Update the car's mileage."""
        if another_variable_in_case >= self.another_variable_in_case:
            self.another_variable_in_case = another_variable_in_case
        else:
            print("Error: another_variable_in_case cannot be reduced.")

    def take_the_bundle_in(self, bundle):
        num_labels, labels, stats, centroids = bundle
        # save the previous iteration
        labels_modified = np.where(labels == self.current_label, self.current_label, 0)
        self.label_history.put(labels_modified)
        self.stats_history.put(self.current_stats)
        self.centroid_history.put(self.current_centroid)
        overlap_labels = queue.Queue()
        # determine which one is the next label
        for i in range(1, num_labels):
            x, y, w, h, area = stats[i]
            if area > min_area:
                overlap_yes = check_overlap(x, y, w, h, self.current_stats[0], self.current_stats[1], self.current_stats[2], self.current_stats[3])
                if overlap_yes:
                    overlap_labels.put(i)
        # find the closest centroid
        centroid_distance = 10000000000 # first assume that the two centroids are very far away
        best_label = -1 # let us not know which one is the best label at the beginning
        for j in range(overlap_labels.qsize()):
            this_label_selected = get_nth_item(overlap_labels, j)
            this_centroid = centroids[this_label_selected]
            this_distance = distance_between_points(this_centroid[0], this_centroid[1], self.current_centroid[0], self.current_centroid[1])
            if this_distance < centroid_distance:
                centroid_distance = this_distance
                best_label = this_label_selected
        # update the current frame
        if best_label == -1: # make sure some labels are selected, if  no overlapping frames exist, jump to the nea
            print('No Criterion satisfied, jump to the nearest frame')
            for l in range(1, num_labels):
                this_centroid = centroids[l]
                this_distance = distance_between_points(this_centroid[0], this_centroid[1], self.current_centroid[0], self.current_centroid[1])
                if this_distance < centroid_distance:
                    centroid_distance = this_distance
                    best_label = l
        self.current_label = best_label
        self.current_stats = stats[best_label]
        self.current_centroid = centroids[best_label]
        self.display_info()

    def write_frame(self, n_th_frame):
        self.original_cap.set(cv2.CAP_PROP_POS_FRAMES, n_th_frame)
        get, this_frame = original_cap.read()
        if not get:
            print('something wrong happened')
        x, y, w, h, area = self.current_stats
        cv2.rectangle(this_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        this_frame = white_outside_rectangle(this_frame, x, y, w, h)
        cv2.imwrite(os.path.join(self.output_folder, f"frame_{n_th_frame}.jpg"), this_frame)

## Frame-by-frame processing

# Process the First Frame

In [19]:
original_cap = cv2.VideoCapture('shorter.mp4')

cap = cv2.VideoCapture('processed_v2.mp4')

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)

erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (75, 75))

# Read the first frame
ret1, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
# Read the second frame
ret2, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
# Read the original frame
ret3, ori_frame = original_cap.read()

# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
prvs = next
cv2.imwrite(os.path.join(monitor_folder, f"frame_{1}.jpg"), rgb_frame)
cv2.imwrite(os.path.join(output_folder, f"frame_{1}.jpg"), ori_frame)

True

# Process the Second Frame

In [20]:
# Read the second frame
ret2, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
# Read the original frame
ret3, ori_frame = original_cap.read()

# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
prvs = next
cv2.imwrite(os.path.join(monitor_folder, f"frame_{2}.jpg"), rgb_frame)
cv2.imwrite(os.path.join(output_folder, f"frame_{2}.jpg"), ori_frame)

True

# Process the Third Frame

In [21]:
# Read the second frame
ret2, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
# Read the original frame
ret3, ori_frame = original_cap.read()

# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
prvs = next
cv2.imwrite(os.path.join(monitor_folder, f"frame_{3}.jpg"), rgb_frame)
cv2.imwrite(os.path.join(output_folder, f"frame_{3}.jpg"), ori_frame)

True

# Set Up the First Worm Object: Worm_1

In [22]:
worm_1 = Worm(4, original_cap, 4, stats[4], centroids[4])
worm_1.display_info()

worn index: 4, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11eeef0d0>, stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

current label: 4, current_stats: [1010  120   86   87 7454], current_centroid: [1052.49946338  163.04292997], stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

Current Stats 

[1010  120   86   87 7454]




Current Label 

4




Current Centroid

[1052.49946338  163.04292997]






## Set up the Worm List

In [23]:
# set up the number that I want to track for worms
track_number = 10

# create the list of n tracking number
w_list = create_list_of_n_objects(track_number)

for i in range(1, len(w_list)):
    temp_Worm = Worm(i, original_cap, i, stats[i], centroids[i])
    w_list[i] = temp_Worm

Check everything in the list

In [24]:
for i in range(1, len(w_list)):
    temp_Worm = Worm(i, original_cap, i, stats[i], centroids[i])
    temp_Worm.display_info()
    w_list[i] = temp_Worm

worn index: 1, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11ef029d0>, stats_history: <queue.Queue object at 0x11ef027d0>, centroid_history: <queue.Queue object at 0x11ef01a90>, 

current label: 1, current_stats: [ 935    0   78   77 6003], current_centroid: [973.48075962  37.98150925], stats_history: <queue.Queue object at 0x11ef027d0>, centroid_history: <queue.Queue object at 0x11ef01a90>, 

Current Stats 

[ 935    0   78   77 6003]




Current Label 

1




Current Centroid

[973.48075962  37.98150925]




worn index: 2, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11eed8050>, stats_history: <queue.Queue object at 0x11eeed510>, centroid_history: <queue.Queue object at 0x11eeef810>, 

current label: 2, current_stats: [1053   12   79   79 6233], current_centroid: [1092.04925397   50.97561367], stats_history: <queue.Queue object at 0x11eeed510>, centroid_history: <queue.Queue object at 0x11eeef810>, 

Curre

# Process the Fourth Frame and take the bundle in by the Worm

In [25]:
# Read the second frame
ret2, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
# Read the original frame
ret3, ori_frame = original_cap.read()

# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
bundle = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)

worm_1.take_the_bundle_in(bundle)
worm_1.write_frame(4)

# process the worm list
for i in range(1, len(w_list)):
    temp_Worm = Worm(i, original_cap, i, stats[i], centroids[i])
    temp_Worm.take_the_bundle_in(bundle)
    temp_Worm.write_frame(4)
    w_list[i] = temp_Worm

for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
prvs = next
cv2.imwrite(os.path.join(monitor_folder, f"frame_{4}.jpg"), rgb_frame)
cv2.imwrite(os.path.join(output_folder, f"frame_{4}.jpg"), ori_frame)

worn index: 4, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11eeef0d0>, stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

current label: 1, current_stats: [ 1026   117   210    97 18763], current_centroid: [1129.28305708  165.21739594], stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

Current Stats 

[ 1026   117   210    97 18763]




Current Label 

1




Current Centroid

[1129.28305708  165.21739594]




worn index: 1, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11ee821d0>, stats_history: <queue.Queue object at 0x11ef0ccd0>, centroid_history: <queue.Queue object at 0x11ef0cdd0>, 

current label: 1, current_stats: [ 1026   117   210    97 18763], current_centroid: [1129.28305708  165.21739594], stats_history: <queue.Queue object at 0x11ef0ccd0>, centroid_history: <queue.Queue object at 0x

True

In [26]:
worm_1.display_info()
print(worm_1.output_folder)

worn index: 4, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11eeef0d0>, stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

current label: 1, current_stats: [ 1026   117   210    97 18763], current_centroid: [1129.28305708  165.21739594], stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

Current Stats 

[ 1026   117   210    97 18763]




Current Label 

1




Current Centroid

[1129.28305708  165.21739594]




/Users/billdeng/PycharmProjects/unicellular/worm_4


# Proceed to the Fifth Frame

In [27]:
# Read the second frame
ret2, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
# Read the original frame
ret3, ori_frame = original_cap.read()

# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
bundle = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)

worm_1.take_the_bundle_in(bundle)
worm_1.write_frame(5)

# process the worm list
for i in range(1, len(w_list)):
    temp_Worm = Worm(i, original_cap, i, stats[i], centroids[i])
    temp_Worm.take_the_bundle_in(bundle)
    temp_Worm.write_frame(5)
    w_list[i] = temp_Worm

for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
prvs = next
cv2.imwrite(os.path.join(monitor_folder, f"frame_{5}.jpg"), rgb_frame)
cv2.imwrite(os.path.join(output_folder, f"frame_{5}.jpg"), ori_frame)

worn index: 4, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11eeef0d0>, stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

current label: 1, current_stats: [1027  119   88   86 7537], current_centroid: [1070.38675866  161.48401221], stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

Current Stats 

[1027  119   88   86 7537]




Current Label 

1




Current Centroid

[1070.38675866  161.48401221]




worn index: 1, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11eecf190>, stats_history: <queue.Queue object at 0x11ef0d850>, centroid_history: <queue.Queue object at 0x11ef0d910>, 

current label: 1, current_stats: [1027  119   88   86 7537], current_centroid: [1070.38675866  161.48401221], stats_history: <queue.Queue object at 0x11ef0d850>, centroid_history: <queue.Queue object at 0x11ef0d910>, 

C

True

In [28]:
worm_1.display_info()
print(worm_1.output_folder)

worn index: 4, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11eeef0d0>, stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

current label: 1, current_stats: [1027  119   88   86 7537], current_centroid: [1070.38675866  161.48401221], stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

Current Stats 

[1027  119   88   86 7537]




Current Label 

1




Current Centroid

[1070.38675866  161.48401221]




/Users/billdeng/PycharmProjects/unicellular/worm_4


## Proceed to the Sixth Frame

In [29]:
# Read the second frame
ret2, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
# Read the original frame
ret3, ori_frame = original_cap.read()

# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
bundle = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)

worm_1.take_the_bundle_in(bundle)
worm_1.write_frame(6)

# process the worm list
for i in range(1, len(w_list)):
    temp_Worm = Worm(i, original_cap, i, stats[i], centroids[i])
    temp_Worm.take_the_bundle_in(bundle)
    temp_Worm.write_frame(6)
    w_list[i] = temp_Worm

for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
prvs = next
cv2.imwrite(os.path.join(monitor_folder, f"frame_{6}.jpg"), rgb_frame)
cv2.imwrite(os.path.join(output_folder, f"frame_{6}.jpg"), ori_frame)

No Criterion satisfied, jump to the nearest frame
worn index: 4, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11eeef0d0>, stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

current label: 1, current_stats: [  425   123   397   267 57098], current_centroid: [617.63856878 248.42047007], stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

Current Stats 

[  425   123   397   267 57098]




Current Label 

1




Current Centroid

[617.63856878 248.42047007]




worn index: 1, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11ee83050>, stats_history: <queue.Queue object at 0x11ef0eb50>, centroid_history: <queue.Queue object at 0x11ef0ec10>, 

current label: 1, current_stats: [  425   123   397   267 57098], current_centroid: [617.63856878 248.42047007], stats_history: <queue.Queue object at 0x11ef0eb50>,

True

In [30]:
worm_1.display_info()
print(worm_1.output_folder)

worn index: 4, video_cap: < cv2.VideoCapture 0x11ee848b0>, label_history: <queue.Queue object at 0x11eeef0d0>, stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

current label: 1, current_stats: [  425   123   397   267 57098], current_centroid: [617.63856878 248.42047007], stats_history: <queue.Queue object at 0x11eeef610>, centroid_history: <queue.Queue object at 0x11eeefbd0>, 

Current Stats 

[  425   123   397   267 57098]




Current Label 

1




Current Centroid

[617.63856878 248.42047007]




/Users/billdeng/PycharmProjects/unicellular/worm_4


## Reset and Process all the frames

# Start with the First Frame

In [31]:
original_cap = cv2.VideoCapture('shorter.mp4')

cap = cv2.VideoCapture('processed_v2.mp4')

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)

erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (75, 75))

# Read the first frame
ret1, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
# Read the second frame
ret2, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
# Read the original frame
ret3, ori_frame = original_cap.read()

# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
prvs = next
cv2.imwrite(os.path.join(monitor_folder, f"frame_{1}.jpg"), rgb_frame)
cv2.imwrite(os.path.join(output_folder, f"frame_{1}.jpg"), ori_frame)

True

# Process the Second Frame

In [32]:
# Read the second frame
ret2, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
# Read the original frame
ret3, ori_frame = original_cap.read()

# Calculate Optical Flow
flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Compute magnitude and angle of the flow
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
# Create motion mask
thresh = 1  # Set threshold for motion detection
motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    if area > min_area:
        cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
prvs = next
cv2.imwrite(os.path.join(monitor_folder, f"frame_{2}.jpg"), rgb_frame)
cv2.imwrite(os.path.join(output_folder, f"frame_{2}.jpg"), ori_frame)

True

# Set up worm list

In [33]:
# set up the number that I want to track for worms
track_number = 10

# create the list of n tracking number
w_list = create_list_of_n_objects(track_number)

for i in range(1, len(w_list)):
    temp_Worm = Worm(i, original_cap, i, stats[i], centroids[i])
    w_list[i] = temp_Worm

# Make sure the list is well set

In [34]:
for i in range(1, len(w_list)):
    temp_Worm = Worm(i, original_cap, i, stats[i], centroids[i])
    temp_Worm.display_info()
    w_list[i] = temp_Worm

worn index: 1, video_cap: < cv2.VideoCapture 0x11ee854b0>, label_history: <queue.Queue object at 0x11eeeeb90>, stats_history: <queue.Queue object at 0x11eec3e90>, centroid_history: <queue.Queue object at 0x11eec0dd0>, 

current label: 1, current_stats: [  924     0   537   250 75010], current_centroid: [1172.90573257  136.73210239], stats_history: <queue.Queue object at 0x11eec3e90>, centroid_history: <queue.Queue object at 0x11eec0dd0>, 

Current Stats 

[  924     0   537   250 75010]




Current Label 

1




Current Centroid

[1172.90573257  136.73210239]




worn index: 2, video_cap: < cv2.VideoCapture 0x11ee854b0>, label_history: <queue.Queue object at 0x11d6b3a90>, stats_history: <queue.Queue object at 0x11eeec250>, centroid_history: <queue.Queue object at 0x11eeefd50>, 

current label: 2, current_stats: [ 810   31  112   89 9733], current_centroid: [865.21103462  75.09339361], stats_history: <queue.Queue object at 0x11eeec250>, centroid_history: <queue.Queue object at 0x11eeefd

# Run the loop to parse the video

In [35]:
for iteration_Number in range(3, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)-1)):

    print(f'\rFrame Number: {iteration_Number} \n', end='')
    # Read the second frame
    ret2, frame2 = cap.read()
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Read the original frame
    ret3, ori_frame = original_cap.read()

    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute magnitude and angle of the flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Create motion mask
    thresh = 1  # Set threshold for motion detection
    motion_mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)[1]
    eroded_frame = cv2.erode(motion_mask, erode_kernel, iterations=1)
    dilated_frame = cv2.dilate(eroded_frame, dilate_kernel, iterations=1)
    bundle = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(dilated_frame), connectivity=8)
    print('Number of Labels', num_labels)
    print('Labels', labels)
    print('stats', stats)
    print('centroid', centroids)
    # process the worm list
    for i in range(1, len(w_list)):
        temp_Worm = Worm(i, original_cap, i, stats[i], centroids[i])
        temp_Worm.take_the_bundle_in(bundle)
        temp_Worm.write_frame(iteration_Number)
        w_list[i] = temp_Worm

    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(dilated_frame, (x, y), (x + w, y + h), (255, 255, 255), 2)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > min_area:
            cv2.rectangle(ori_frame, (x, y), (x + w, y + h), (250,128,114), 2)
    temp_frame = cv2.normalize(dilated_frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    rgb_frame = cv2.cvtColor(temp_frame, cv2.COLOR_GRAY2RGB)
    prvs = next
    cv2.imwrite(os.path.join(monitor_folder, f"frame_{iteration_Number}.jpg"), rgb_frame)
    cv2.imwrite(os.path.join(output_folder, f"frame_{iteration_Number}.jpg"), ori_frame)

Frame Number: 3 
Number of Labels 23
Labels [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
stats [[      0       0    2160    2160 3893072]
 [    935       0      78      77    6003]
 [   1053      12      79      79    6233]
 ...
 [    236    1591     139     121   15414]
 [    376    1638     225     234   41513]
 [    819    1746     742     359  135367]]
centroid [[1077.21786831 1037.29386664]
 [ 973.48075962   37.98150925]
 [1092.04925397   50.97561367]
 [1162.04863222  153.05471125]
 [1052.49946338  163.04292997]
 [ 574.51002801  280.78412206]
 [1314.5         195.5       ]
 [ 194.49367089  500.51898734]
 [ 231.24426824  633.56358719]
 [1964.71074801  741.59680263]
 [ 786.54372894  874.09124046]
 [1091.62124831  901.54975718]
 [2028.20329935  978.70038945]
 [ 127.90485275  979.32025754]
 [ 522.45516024 1071.90412143]
 [1551.01466915 1174.70608928]
 [1978.05242965 1268.70604034]
 [1065.22762008 1306.94187176]

IndexError: index 1 is out of bounds for axis 0 with size 1